In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [9]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]


In [10]:
train = train.where(train["game_time"] < 10000000)
train = train.drop("match_id", axis = 1)
train = train.dropna()

In [11]:
from sklearn.model_selection import train_test_split
random_seed = 42
raw_train, raw_test = train_test_split(train, test_size=0.2, random_state=random_seed)
Y_train = pd.DataFrame(raw_train['target'])
X_train = raw_train.drop('target', axis = 1)
Y_test = pd.DataFrame(raw_test['target'])
X_test = raw_test.drop('target', axis = 1)
Y_train["target"] = Y_train['target'].astype(int)
Y_test["target"] = Y_test['target'].astype(int)

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {
    'C' : [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100],
    'gamma' : [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100],
    'kernel' : ['rbf']
}
svc = SVC()
model = GridSearchCV(svc, parameters, cv = 5, scoring= 'roc_auc')
model.fit(X_train, Y_train)
model.best_params_

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConve

{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}

In [13]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.5
ROC-AUC score : 0.5


In [14]:
from write import write
write(model)